# The visuals

This is the notebook with all the visuals from the initial data scrapping and wrangling for the "Can you win anything with kids?" article in WAGNH.

## Initial imports

In [ ]:
import csv
import random
import time
import os
import re
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!cd

## Get the data

In [ ]:
season_summaries_dir = 'season_summaries'

season_dataframes = []

for filename in os.listdir(season_summaries_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(season_summaries_dir, filename)
        season_df = pd.read_csv(file_path)
        season_dataframes.append(season_df)

df = pd.concat(season_dataframes, ignore_index=True)
df = df[df['Season'].str.contains('2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022|2023')]

## Exploratory data analysis

### Assessing dataframe structure

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

### Getting unique counts

In [ ]:
unique_player_count = df['Player Name'].nunique()
print(f"Unique number of players: {unique_player_count}")

In [ ]:
unique_players_by_position = df.groupby('Player Position')['Player Name'].nunique()
print(unique_players_by_position)

### Correlation

In [ ]:
corr = df.corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr, annot=True, fmt=".2f")

In [ ]:
correlation = df['League Position'].corr(df['Top Competition Starts Before Season'])
print(f"Correlation coefficient: {correlation}")

In [ ]:
correlation = df['League Position'].corr(df['Top Competition Appearances Before Season'])
print(f"Correlation coefficient: {correlation}")

### Distribution plots

In [ ]:
plt.figure(figsize=(12, 8))

histplot = sns.histplot(df['Age at End of Season'], bins=27, kde=False)

for p in histplot.patches:
    height = p.get_height()
    histplot.text(x=p.get_x() + p.get_width() / 2, y=height + 5, s=f'{int(height)}', ha='center')

plt.title('Distribution of Age at End of Season')
plt.xlabel('Age at End of Season')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize=(14, 6))
displot1 = sns.displot(df['Total Starts Before Season'], bins=42, kde=False, height=6, aspect=2.33)
for bar in displot1.ax.patches:
    displot1.ax.annotate(format(bar.get_height()), 
                   (bar.get_x() + bar.get_width() / 2, bar.get_height()), 
                   ha='center', va='center',
                   size=9.5, xytext=(0.5, 8),
                   textcoords='offset points')
plt.title('Distribution of Total Starts Before Season')

In [ ]:
plt.figure(figsize=(14, 6))
displot2 = sns.displot(df['Top Competition Starts Before Season'], bins=41, kde=False, height=6, aspect=2.33)
for bar in displot2.ax.patches:
    displot2.ax.annotate(format(bar.get_height()), 
                   (bar.get_x() + bar.get_width() / 2, bar.get_height()), 
                   ha='center', va='center',
                   size=9.5, xytext=(0.5, 8),
                   textcoords='offset points')
plt.title('Distribution of Top Competition Starts Before Season')

### Boxplots

In [ ]:
sns.boxplot(x='Role', y='Age at End of Season', data=df)

In [ ]:
sns.boxplot(x='Role', y='Total Starts Before Season', data=df)

In [ ]:
sns.boxplot(x='Role', y='Top Competition Starts Before Season', data=df)

In [ ]:
sns.barplot(x='Role', y='Total Starts Before Season', data=df)

### Barplots

In [ ]:
sns.barplot(x='Role', y='Top Competition Starts Before Season', data=df)

### League grouped analysis

In [ ]:
grouped_data = df.groupby('League Position')['Total Starts Before Season'].mean().reset_index()
grouped_data.sort_values(by='League Position', inplace=True)

In [ ]:
sns.barplot(x='League Position', y='Total Starts Before Season', data=grouped_data)
plt.title('Average Total Starts by League Position')
plt.xlabel('League Position')
plt.ylabel('Average Total Starts Before Season')

In [ ]:
grouped_data = df.groupby('League Position')['Top Competition Starts Before Season'].mean().reset_index()
grouped_data.sort_values(by='League Position', inplace=True)

In [ ]:
sns.barplot(x='League Position', y='Top Competition Starts Before Season', data=grouped_data)
plt.title('Average Top Competition Starts by League Position')
plt.xlabel('League Position')
plt.ylabel('Average Top Competition Starts Before Season')

In [ ]:
sns.boxplot(x='League Position', y='Total Starts Before Season', data=df)
plt.title('Total Starts Before Season by League Position')
plt.xlabel('League Position')
plt.ylabel('Total Starts Before Season')

In [ ]:
sns.boxplot(x='League Position', y='Top Competition Starts Before Season', data=df)
plt.title('Top Competition Starts Before Season by League Position')
plt.xlabel('League Position')
plt.ylabel('Top Competition Starts Before Season')

## The teams

### Unique teams

In [ ]:
unique_teams = df['Team'].unique()
teams_df = pd.DataFrame(unique_teams, columns=['Team'])
print(teams_df)

In [ ]:
teams_array = teams_df['Team'].values.reshape(10, 4)
reshaped_teams_df = pd.DataFrame(teams_array, columns=['PL Teams','PL Teams','PL Teams','PL Teams'])

output_directory = 'analysis_output\\csv'
os.makedirs(output_directory, exist_ok=True)

csv_file_path = os.path.join(output_directory, 'unique_teams_2008-2023.csv')
reshaped_teams_df.to_csv(csv_file_path, index=False)

print(f"Saved reshaped DataFrame to {csv_file_path}")

In [ ]:
unique_seasons_set = set(df['Season'].unique())

teams_in_all_seasons = []

for team in df['Team'].unique():
    team_seasons_set = set(df[df['Team'] == team]['Season'].unique())

    if team_seasons_set == unique_seasons_set:
        teams_in_all_seasons.append(team)

teams_in_all_seasons_df = pd.DataFrame(teams_in_all_seasons, columns=['Team'])

print(teams_in_all_seasons_df)

### League averages

In [ ]:
df['League Position'] = pd.to_numeric(df['League Position'], errors='coerce')
average_positions = df.groupby('Team')['League Position'].mean().sort_values()

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(x=average_positions.values, y=average_positions.index)

plt.title('Average League Position of Premier League Teams (2008-2023)')
plt.xlabel('Average Position')
plt.ylabel('Team')

In [ ]:
plt.figure(figsize=(12, 10))
sns_barplot = sns.barplot(x=average_positions.values, y=average_positions.index)

plt.title('Average League Position of Premier League Teams (2008-2023)')
plt.xlabel('Average Position')
plt.ylabel('Team')

for p in sns_barplot.patches:
    width = p.get_width()
    sns_barplot.text(width + 0.1,
                     p.get_y() + p.get_height() / 2,
                     '{:1.2f}'.format(width),
                     ha = 'center',
                     va = 'center')

In [ ]:
league_tables_dir = 'league_table'
all_seasons_df = []

for filename in os.listdir(league_tables_dir):
    if filename.endswith('.csv'):
        season = filename.split('_')[3].split('.')[0]
        start_year = int(season.split('-')[0])
        if 2008 <= start_year <= 2022:
            season_df = pd.read_csv(os.path.join(league_tables_dir, filename))
            season_df[['Goals For', 'Goals Against']] = season_df['Goals'].str.split(':', expand=True).iloc[:, :2]
            season_df['Goals For'] = pd.to_numeric(season_df['Goals For'], errors='coerce')
            season_df['Goals Against'] = pd.to_numeric(season_df['Goals Against'], errors='coerce')
            season_df['Points'] = pd.to_numeric(season_df['Points'], errors='coerce')
            season_df['Season'] = season
            all_seasons_df.append(season_df)

league_df = pd.concat(all_seasons_df, ignore_index=True)

league_df = league_df[['Team', 'Points', 'Goals For', 'Goals Against', 'Season']]

team_averages = league_df.groupby('Team').mean()

In [ ]:
team_averages = team_averages.sort_values('Points', ascending=False)

plt.figure(figsize=(12, 10))

sns_barplot = sns.barplot(x=team_averages['Points'].values, y=team_averages.index)
plt.title('Average Points of Premier League Teams (2008-2023)')
plt.xlabel('Average Points')
plt.ylabel('Team')

for p in sns_barplot.patches:
    width = p.get_width()
    plt.text(width + 0.5,
             p.get_y() + p.get_height() / 2,
             f'{width:.1f}',
             va='center')

In [ ]:
team_goals_averages = league_df.groupby('Team')[['Goals For', 'Goals Against']].mean()

In [ ]:
team_goals_averages_sorted_gf = team_goals_averages.sort_values('Goals For', ascending=False)

plt.figure(figsize=(12, 10))
sns_barplot_gf = sns.barplot(x=team_goals_averages_sorted_gf['Goals For'].values, y=team_goals_averages_sorted_gf.index)

for p in sns_barplot_gf.patches:
    width = p.get_width()
    plt.text(width + 0.1, p.get_y() + p.get_height() / 2, f'{width:.1f}', va='center')

plt.title('Average Goals Scored by Premier League Teams (2008-2023)')
plt.xlabel('Average Goals Scored')
plt.ylabel('Team')

In [ ]:
team_goals_averages_sorted_ga = team_goals_averages.sort_values('Goals Against', ascending=True)

plt.figure(figsize=(12, 10))
sns_barplot_ga = sns.barplot(x=team_goals_averages_sorted_ga['Goals Against'].values, y=team_goals_averages_sorted_ga.index)

for p in sns_barplot_ga.patches:
    width = p.get_width()
    plt.text(width + 0.1, p.get_y() + p.get_height() / 2, f'{width:.1f}', va='center')

plt.title('Average Goals Conceded by Premier League Teams (2008-2023)')
plt.xlabel('Average Goals Conceded')
plt.ylabel('Team')

### Overall age and experience

In [ ]:
df_sorted = df.sort_values(by=['Player URL', 'Season'])
df_first_age = df_sorted.drop_duplicates(subset='Player URL', keep='first')
df_final_age = df_sorted.drop_duplicates(subset='Player URL', keep='last')

In [ ]:
plt.figure(figsize=(12, 8))
histplot_first = sns.histplot(df_first_age['Age at End of Season'], bins=25, kde=False)
for p in histplot_first.patches:
    height = p.get_height()
    histplot_first.text(x=p.get_x() + p.get_width() / 2, y=height + 5, s=f'{int(height)}', ha='center')
plt.title('Distribution of Initial Age in the Premier League')
plt.xlabel('First Age at End of Season')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize=(12, 8))
histplot_final = sns.histplot(df_final_age['Age at End of Season'], bins=27, kde=False)
for p in histplot_final.patches:
    height = p.get_height()
    histplot_final.text(x=p.get_x() + p.get_width() / 2, y=height + 5, s=f'{int(height)}', ha='center')
plt.title('Distribution of Final Age in the Premier League')
plt.xlabel('Final Age at End of Season')
plt.ylabel('Count')

In [ ]:
df_sorted = df.sort_values(by=['Player URL', 'Season'])
df_initial_experience = df_sorted.drop_duplicates(subset='Player URL', keep='first')

In [ ]:
plt.figure(figsize=(16, 8))
histplot_total_starts_initial = sns.histplot(df_initial_experience['Total Starts Before Season'], bins=39, kde=False)

offset = 0.3

for p in histplot_total_starts_initial.patches:
    height = p.get_height()
    histplot_total_starts_initial.text(x=p.get_x() + p.get_width() / 2 + offset, y=height + 5, s=f'{int(height)}', ha='center')

plt.title('Initial Total Starts Before Season for Each Player')
plt.xlabel('Total Starts Before Season')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize=(12, 8))
histplot_top_starts_initial = sns.histplot(df_initial_experience['Top Competition Starts Before Season'], bins=35, kde=False)
for p in histplot_top_starts_initial.patches:
    height = p.get_height()
    histplot_top_starts_initial.text(x=p.get_x() + p.get_width() / 2, y=height + 5, s=f'{int(height)}', ha='center')
plt.title('Initial Top Competition Starts Before Season for Each Player')
plt.xlabel('Top Competition Starts Before Season')
plt.ylabel('Count')

In [ ]:
df_final_experience = df_sorted.drop_duplicates(subset='Player URL', keep='last')

In [ ]:
plt.figure(figsize=(12, 8))
histplot_total_starts_final = sns.histplot(df_final_experience['Total Starts Before Season'], bins=50, kde=False)
for p in histplot_total_starts_final.patches:
    height = p.get_height()
    histplot_total_starts_final.text(x=p.get_x() + p.get_width() / 2, y=height + 5, s=f'{int(height)}', ha='center')
plt.title('Final Total Starts Before Season for Each Player')
plt.xlabel('Total Starts Before Season')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize=(16, 10))
histplot_top_starts_final = sns.histplot(df_final_experience['Top Competition Starts Before Season'], bins=50, kde=False)
for p in histplot_top_starts_final.patches:
    height = p.get_height()
    histplot_top_starts_final.text(x=p.get_x() + p.get_width() / 2, y=height + 5, s=f'{int(height)}', ha='center')
plt.title('Final Top Competition Starts Before Season for Each Player')
plt.xlabel('Top Competition Starts Before Season')
plt.ylabel('Count')

## The players

### Unique players

In [ ]:
df.shape

In [ ]:
unique_player_count = df['Player Name'].nunique()
print(f"Unique number of players: {unique_player_count}")

In [ ]:
unique_players_by_position = df.groupby('Player Position')['Player Name'].nunique()
print(unique_players_by_position)

### The average player

In [ ]:
player_career_data = df.groupby('Player URL').agg(
    Total_PL_Starts=('PL Starts This Season', 'sum'),
    Total_PL_Appearances=('PL Appearances This Season', 'sum'),
    Total_Seasons=('Season', 'nunique')
)

average_starts = player_career_data['Total_PL_Starts'].mean()
average_appearances = player_career_data['Total_PL_Appearances'].mean()
average_seasons = player_career_data['Total_Seasons'].mean()

print(f"Average Premier League Starts: {average_starts:.2f}")
print(f"Average Premier League Appearances: {average_appearances:.2f}")
print(f"Average Number of Seasons in Premier League: {average_seasons:.2f}")

### Top appearances and starts

In [ ]:
player_totals = df.groupby('Player Name').agg(
    Total_PL_Appearances=('PL Appearances This Season', 'sum'),
    Total_PL_Starts=('PL Starts This Season', 'sum')
).reset_index()

In [ ]:
top_appearances_players = player_totals.sort_values(by='Total_PL_Appearances', ascending=False).head(10)['Player Name']

top_starts_players = player_totals.sort_values(by='Total_PL_Starts', ascending=False).head(10)['Player Name']

In [ ]:
df_top_appearances_info = df[df['Player Name'].isin(top_appearances_players)]

df_top_starts_info = df[df['Player Name'].isin(top_starts_players)]

In [ ]:
def summarize_player_appearances(df):
    summarized_df = df.groupby('Player Name').agg({
        'Team': lambda x: ', '.join(sorted(set(x))),
        'Season': lambda x: f'{min(x, key=lambda s: int(s.split("-")[0]))}/{max(x, key=lambda s: int(s.split("-")[1]))}',
        'PL Appearances This Season': 'sum'
    }).reset_index()
    return summarized_df

In [ ]:
def summarize_player_starts(df):
    summarized_df = df.groupby('Player Name').agg({
        'Team': lambda x: ', '.join(sorted(set(x))),
        'Season': lambda x: f'{min(x, key=lambda s: int(s.split("-")[0]))}/{max(x, key=lambda s: int(s.split("-")[1]))}',
        'PL Starts This Season': 'sum'
    }).reset_index()
    return summarized_df

In [ ]:
summarized_top_appearances = summarize_player_appearances(df_top_appearances_info)
summarized_top_appearances = summarized_top_appearances.sort_values(by='PL Appearances This Season', ascending=False).reset_index(drop=True)

summarized_top_starts = summarize_player_starts(df_top_starts_info)
summarized_top_starts = summarized_top_starts.sort_values(by='PL Starts This Season', ascending=False).reset_index(drop=True)

In [ ]:
output_dir = 'analysis_output\\csv'
os.makedirs(output_dir, exist_ok=True)

summarized_top_appearances.to_csv(os.path.join(output_dir, 'top_appearances_players.csv'), index=False)
summarized_top_starts.to_csv(os.path.join(output_dir, 'top_starts_players.csv'), index=False)

### Top goals

In [ ]:
player_name_mapping = {}
season_summaries_dir = 'season_summaries'
for summary_filename in os.listdir(season_summaries_dir):
    if summary_filename.endswith('.csv') and not summary_filename.startswith('2023-2024'):
        summary_filepath = os.path.join(season_summaries_dir, summary_filename)
        summary_df = pd.read_csv(summary_filepath)
        for index, row in summary_df.iterrows():
            player_folder = row['Player URL'].rstrip('/').split('/')[-1]
            player_name_mapping[player_folder] = row['Player Name']

In [ ]:
def is_valid_season_format(season):
    return bool(re.match(r'^\d{4}/\d{4}$', season))

In [ ]:
player_goals_list = []
top_players_dir = 'top_players'

if os.path.exists(top_players_dir):
    for player_folder in os.listdir(top_players_dir):
        player_path = os.path.join(top_players_dir, player_folder)
        career_data_file = os.path.join(player_path, f"{player_folder}_career_data.csv")
        if os.path.exists(career_data_file):
            player_stats = pd.read_csv(career_data_file)
            player_stats['Season'] = player_stats['Season'].astype(str)
            pl_stats = player_stats[
                (player_stats['League'] == 'Pr. League') &
                (player_stats['Season'].apply(is_valid_season_format))
            ]
            pl_stats = pl_stats[pl_stats['Season'].apply(lambda s: 2008 <= int(s.split('/')[0]) <= 2023)]
            
            if not pl_stats.empty:
                total_goals = pl_stats['Goals'].sum()
                teams_played_for = pl_stats['Team'].unique().tolist()
                seasons_played = pl_stats['Season'].unique()

                if seasons_played.size > 0:
                    season_start_min = min(seasons_played, key=lambda s: int(s.split('/')[0]))
                    season_end_max = max(seasons_played, key=lambda s: int(s.split('/')[1]))
                    season_range = f"{season_start_min.split('/')[0]}-{season_start_min.split('/')[1]}/{season_end_max.split('/')[0]}-{season_end_max.split('/')[1]}"
                else:
                    season_range = 'N/A'

                player_name = player_name_mapping.get(player_folder, player_folder)
                player_goals_list.append({
                    'Player Name': player_name,
                    'Teams': ', '.join(teams_played_for),
                    'Seasons': season_range,
                    'Goals': total_goals
                })
else:
    print(f"The directory {top_players_dir} does not exist.")

In [ ]:
player_goals_df = pd.DataFrame(player_goals_list)

player_goals_df_sorted = player_goals_df.sort_values(by='Goals', ascending=False)

In [ ]:
output_dir = 'analysis_output/csv'
os.makedirs(output_dir, exist_ok=True)
output_filepath = os.path.join(output_dir, 'top_goalscorers.csv')
player_goals_df_sorted.to_csv(output_filepath, index=False)

print(f"Top goalscorers data saved to {output_filepath}")

In [ ]:
top_10_goalscorers = player_goals_df_sorted = player_goals_df.sort_values(by='Goals', ascending=False).head(10)

In [ ]:
output_directory = 'analysis_output\\csv'
output_file = 'top_10_premier_league_goalscorers.csv'

os.makedirs(output_directory, exist_ok=True)

output_path = os.path.join(output_directory, output_file)
top_10_goalscorers.to_csv(output_path, index=False)

print(f"Top 10 Premier League goalscorers saved to {output_path}")

### Average league position

In [ ]:
def format_season_range(seasons):
    years = []
    for season in seasons:
        start_year, end_year = season.split('-')
        start_year = int(start_year)
        end_year = int(end_year)
        years.append((start_year, end_year))
    
    min_start_year = min(years, key=lambda x: x[0])[0]
    max_end_year = max(years, key=lambda x: x[1])[1]

    return f"{min_start_year}-{min_start_year + 1}/{max_end_year}-{max_end_year + 1}"

In [ ]:
df['PL Appearances This Season'] = pd.to_numeric(df['PL Appearances This Season'], errors='coerce')

players_over_200_appearances = df.groupby('Player Name')['PL Appearances This Season'].sum()
players_over_200_appearances = players_over_200_appearances[players_over_200_appearances > 200]

df_filtered = df[df['Player Name'].isin(players_over_200_appearances.index)]

df_filtered['League Position'] = pd.to_numeric(df_filtered['League Position'], errors='coerce')
player_avg_positions = df_filtered.groupby('Player Name')['League Position'].mean().reset_index()

player_avg_positions['League Position'] = player_avg_positions['League Position'].round(2)

player_additional_info = df_filtered.groupby('Player Name').agg({
    'Player Position': 'first',
    'Season': lambda x: format_season_range(sorted(set(x))),
    'Team': lambda x: ', '.join(sorted(set(x)))
}).reset_index()

merged_df = pd.merge(player_avg_positions, player_additional_info, on='Player Name')

merged_df = merged_df.sort_values(by='League Position', ascending=True)

top_20_players_avg_position = merged_df.head(20)

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)

complete_list_file = os.path.join(output_directory, 'complete_list_avg_league_positions.csv')
merged_df.to_csv(complete_list_file, index=False)

top_20_list_file = os.path.join(output_directory, 'top_20_players_avg_league_positions.csv')
top_20_players_avg_position.to_csv(top_20_list_file, index=False)

print(f"Complete list saved to {complete_list_file}")
print(f"Top 10 list saved to {top_20_list_file}")

### The positional averages

In [ ]:
average_stats_by_position = df.groupby('Player Position').agg({
    'Age at End of Season': 'mean',
    'Top Competition Starts Before Season': 'mean',
    'Top Competition Appearances Before Season': 'mean',
    'Total Starts Before Season': 'mean',
    'Total Appearances Before Season': 'mean'
}).reset_index()

average_stats_by_position = average_stats_by_position.round(0)

print(average_stats_by_position)

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

average_stats_by_position = df.groupby('Player Position').agg({
    'Age at End of Season': 'mean',
    'Top Competition Starts Before Season': 'mean',
    'Top Competition Appearances Before Season': 'mean',
    'Total Starts Before Season': 'mean',
    'Total Appearances Before Season': 'mean'
}).reset_index()

average_stats_by_position = average_stats_by_position.round(0).astype({'Age at End of Season': int, 
                                                                       'Top Competition Starts Before Season': int, 
                                                                       'Top Competition Appearances Before Season': int, 
                                                                       'Total Starts Before Season': int, 
                                                                       'Total Appearances Before Season': int})

average_stats_by_position['Player Position'] = pd.Categorical(average_stats_by_position['Player Position'], categories=position_order, ordered=True)

average_stats_by_position = average_stats_by_position.sort_values('Player Position')

In [ ]:
average_stats_by_position

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)
output_file_path = os.path.join(output_directory, 'positional_averages.csv')
average_stats_by_position.to_csv(output_file_path, index=False)

print(f"Positional averages saved to {output_file_path}")

#### Role breakdown

In [ ]:
role_counts_by_position = df.groupby(['Player Position', 'Role']).size().unstack(fill_value=0)

total_role_counts = df['Role'].value_counts()

In [ ]:
colors = ['blue', 'green', 'red']

for position in role_counts_by_position.index:
    role_counts = role_counts_by_position.loc[position]
    role_counts.plot(kind='bar', figsize=(10, 6), rot=0, color=colors)
    plt.title(f'Role Breakdown for {position}')
    plt.xlabel('Role')
    plt.ylabel('Count of Players')
    plt.xticks([0, 1, 2], ['1st Choice', '2nd Choice', '3rd Choice'])
    plt.show()

In [ ]:
role_counts = df.groupby(['Player Position', 'Role']).size().reset_index(name='Count')

total_by_position = role_counts.groupby('Player Position')['Count'].transform('sum')

role_counts['Percentage'] = (role_counts['Count'] / total_by_position) * 100

role_percentages = role_counts.pivot(index='Player Position', columns='Role', values='Percentage')

print(role_percentages)

In [ ]:
colors = ['blue', 'green', 'red']

for position in role_percentages.index:
    role_percentage = role_percentages.loc[position]
    role_percentage.plot(kind='bar', figsize=(10, 6), rot=0, color=colors)
    plt.title(f'Role Percentage Breakdown for {position}')
    plt.xlabel('Role')
    plt.ylabel('Percentage')
    plt.xticks([0, 1, 2], ['1st Choice', '2nd Choice', '3rd Choice'])
    plt.show()

In [ ]:
reshaped_df = df.groupby(['Player Position', 'Role']).size().reset_index(name='Count')
reshaped_df = reshaped_df.pivot(index='Player Position', columns='Role', values='Count').fillna(0)

plt.figure(figsize=(12, 8))
reshaped_df.plot(kind='bar', stacked=True)
plt.title('Combined Role Breakdown Across Positions')
plt.xlabel('Player Position')
plt.ylabel('Count of Players')
plt.xticks(rotation=45)
plt.legend(title='Role')

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
role_percentages = role_percentages.reindex(position_order)

plt.figure(figsize=(16, 10))
bar_width = 0.25
index = np.arange(len(position_order))

for i, role in enumerate(role_percentages.columns):
    plt.bar(index + i * bar_width, role_percentages[role], bar_width, label=role)

plt.xlabel('Player Position')
plt.ylabel('Percentage')
plt.title('Percentage Role Breakdown Across Positions')
plt.xticks(index + bar_width, position_order)
plt.legend(title='Role')

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']
role_percentages = role_percentages.reindex(position_order)

plt.figure(figsize=(16, 10))
bar_width = 0.25
index = np.arange(len(position_order))

bars = []

for i, role in enumerate(role_percentages.columns):
    bar = plt.bar(index + i * bar_width, role_percentages[role], bar_width, label=role)
    bars.append(bar)

plt.xlabel('Player Position')
plt.ylabel('Percentage')
plt.title('Percentage Role Breakdown Across Positions')
plt.xticks(index + bar_width, position_order)
plt.legend(title='Role')

for bar in bars:
    for rect in bar:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.1f}%', ha='center', va='bottom')

## ...and can you win anything with kids?

### Top six teams

In [ ]:
avg_league_positions = df.groupby('Team')['League Position'].mean().sort_values().reset_index()

top_six_teams = avg_league_positions.head(6)['Team'].tolist()

In [ ]:
top_six_teams

In [ ]:
df_top_six = df[df['Team'].isin(top_six_teams)]

In [ ]:
df_top_six

In [ ]:
df_top_six.columns

### Average ages in the top six

In [ ]:
avg_age_by_team = df_top_six.groupby('Team')['Age at End of Season'].mean().reset_index()
avg_age_by_team.rename(columns={'Age at End of Season': 'Average Age'}, inplace=True)

In [ ]:
first_appearance = df_top_six.groupby(['Player Name', 'Team'])['Season'].min().reset_index()
first_appearance = pd.merge(first_appearance, df_top_six[['Player Name', 'Season', 'Age at End of Season']], 
                            left_on=['Player Name', 'Season'], right_on=['Player Name', 'Season'])
entry_age_by_team = first_appearance.groupby('Team')['Age at End of Season'].mean().reset_index()
entry_age_by_team.rename(columns={'Age at End of Season': 'Entry Age'}, inplace=True)

In [ ]:
last_appearance = df_top_six.groupby(['Player Name', 'Team'])['Season'].max().reset_index()
last_appearance = pd.merge(last_appearance, df_top_six[['Player Name', 'Season', 'Age at End of Season']], 
                           left_on=['Player Name', 'Season'], right_on=['Player Name', 'Season'])
exit_age_by_team = last_appearance.groupby('Team')['Age at End of Season'].mean().reset_index()
exit_age_by_team.rename(columns={'Age at End of Season': 'Exit Age'}, inplace=True)

In [ ]:
age_overview = pd.merge(avg_age_by_team, entry_age_by_team, on='Team')
age_overview = pd.merge(age_overview, exit_age_by_team, on='Team')

In [ ]:
age_overview

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
age_overview['Average Age'] = age_overview['Average Age'].round(2)
age_overview['Entry Age'] = age_overview['Entry Age'].round(2)
age_overview['Exit Age'] = age_overview['Exit Age'].round(2)

In [ ]:
age_overview = age_overview.sort_values('Average Age', ascending=False)

In [ ]:
age_overview

In [ ]:
age_overview_file = os.path.join(output_directory, 'age_overview.csv')
age_overview.to_csv(age_overview_file, index=False)
print(f"Age overview saved to {age_overview_file}")

### League position vs. amount of experience

In [ ]:
avg_league_position = df_top_six.groupby('Team')['League Position'].mean().reset_index()
ordered_teams = avg_league_position.sort_values('League Position')['Team']

In [ ]:
experience_metrics = df_top_six.groupby('Team').agg({
    'Top Competition Starts Before Season': 'mean',
    'Top Competition Appearances Before Season': 'mean',
    'Total Starts Before Season': 'mean',
    'Total Appearances Before Season': 'mean'
}).reset_index()

experience_metrics_ordered = experience_metrics.set_index('Team').loc[ordered_teams].reset_index()

In [ ]:
plt.figure(figsize=(14, 8))
bar_width = 0.35
index = np.arange(len(ordered_teams))

plt.bar(index, experience_metrics_ordered['Top Competition Starts Before Season'], bar_width, label='Top Competition Starts')
plt.bar(index + bar_width, experience_metrics_ordered['Top Competition Appearances Before Season'], bar_width, label='Top Competition Appearances')

offset = max(experience_metrics_ordered['Top Competition Starts Before Season'].max(), experience_metrics_ordered['Top Competition Appearances Before Season'].max()) * 0.01
for i in range(len(ordered_teams)):
    plt.text(i, experience_metrics_ordered['Top Competition Starts Before Season'][i] + offset, f'{experience_metrics_ordered["Top Competition Starts Before Season"][i]:.2f}', ha='center')
    plt.text(i + bar_width, experience_metrics_ordered['Top Competition Appearances Before Season'][i] + offset, f'{experience_metrics_ordered["Top Competition Appearances Before Season"][i]:.2f}', ha='center')

plt.xlabel('Team')
plt.ylabel('Experience')
plt.title('Top Competition Experience Metrics of Top Six Teams')
plt.xticks(index + bar_width / 2, ordered_teams, rotation=45)
plt.legend()

In [ ]:
plt.figure(figsize=(14, 8))

plt.bar(index, experience_metrics_ordered['Total Starts Before Season'], bar_width, label='Total Starts')
plt.bar(index + bar_width, experience_metrics_ordered['Total Appearances Before Season'], bar_width, label='Total Appearances')

offset = max(experience_metrics_ordered['Total Starts Before Season'].max(), experience_metrics_ordered['Total Appearances Before Season'].max()) * 0.01  
for i in range(len(ordered_teams)):
    plt.text(i, experience_metrics_ordered['Total Starts Before Season'][i] + offset, f'{experience_metrics_ordered["Total Starts Before Season"][i]:.2f}', ha='center')
    plt.text(i + bar_width, experience_metrics_ordered['Total Appearances Before Season'][i] + offset, f'{experience_metrics_ordered["Total Appearances Before Season"][i]:.2f}', ha='center')

plt.xlabel('Team')
plt.ylabel('Experience')
plt.title('Total Experience Metrics of Top Six Teams')
plt.xticks(index + bar_width / 2, ordered_teams, rotation=45)
plt.legend()

### Squad role vs. amount of experience per position

In [ ]:
average_experience_by_role_position = df_top_six.groupby(['Player Position', 'Role']).agg({
    'Top Competition Starts Before Season': 'mean',
    'Top Competition Appearances Before Season': 'mean',
    'Total Starts Before Season': 'mean',
    'Total Appearances Before Season': 'mean'
}).reset_index()

average_experience_by_role_position = average_experience_by_role_position.round(2)

print(average_experience_by_role_position)

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

plt.figure(figsize=(12, 8))

barplot = sns.barplot(
    data=average_experience_by_role_position, 
    x='Player Position', 
    y='Top Competition Starts Before Season', 
    hue='Role',
    order=position_order
)

for p in barplot.patches:
    height = p.get_height()
    if not np.isnan(height):
        barplot.annotate(format(height, '.2f'), 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                        textcoords = 'offset points')

plt.title('Average Top Competition Starts Before Season by Role and Position')
plt.xlabel('Player Position')
plt.ylabel('Average Starts')
plt.legend(title='Role')
plt.xticks(rotation=45)

In [ ]:
plt.figure(figsize=(12, 8))

barplot = sns.barplot(
    data=average_experience_by_role_position, 
    x='Player Position', 
    y='Top Competition Appearances Before Season', 
    hue='Role',
    order=position_order
)

for p in barplot.patches:
    height = p.get_height()
    if not np.isnan(height):  # Check for NaN values to avoid errors
        barplot.annotate(format(height, '.2f'), 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                        textcoords = 'offset points')

plt.title('Average Top Competition Appearances Before Season by Role and Position')
plt.xlabel('Player Position')
plt.ylabel('Average Appearances')
plt.legend(title='Role')
plt.xticks(rotation=45)

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

plt.figure(figsize=(12, 8))

barplot = sns.barplot(
    data=average_experience_by_role_position, 
    x='Player Position', 
    y='Total Starts Before Season', 
    hue='Role',
    order=position_order
)

for p in barplot.patches:
    height = p.get_height()
    if not np.isnan(height):
        barplot.annotate(format(height, '.2f'), 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                        textcoords = 'offset points')

plt.title('Average Total Starts Before Season by Role and Position')
plt.xlabel('Player Position')
plt.ylabel('Average Starts')
plt.legend(title='Role')
plt.xticks(rotation=45)

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

plt.figure(figsize=(12, 8))

barplot = sns.barplot(
    data=average_experience_by_role_position, 
    x='Player Position', 
    y='Total Appearances Before Season', 
    hue='Role',
    order=position_order
)

for p in barplot.patches:
    height = p.get_height()
    if not np.isnan(height):
        barplot.annotate(format(height, '.2f'), 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                        textcoords = 'offset points')

plt.title('Average Total Appearances Before Season by Role and Position')
plt.xlabel('Player Position')
plt.ylabel('Average Starts')
plt.legend(title='Role')
plt.xticks(rotation=45)

#### Min-max experience (not filtered)

In [ ]:
def min_max_with_season_player(df, column):
    min_value = df[column].min()
    max_value = df[column].max()
    min_records = df[df[column] == min_value][['Player Name', 'Season']]
    max_records = df[df[column] == max_value][['Player Name', 'Season']]
    
    min_info = '; '.join([f"{row['Player Name']} ({row['Season']})" for _, row in min_records.iterrows()])
    max_info = '; '.join([f"{row['Player Name']} ({row['Season']})" for _, row in max_records.iterrows()])

    return pd.Series({
        f'{column}_min': min_value,
        f'{column}_min_info': min_info,
        f'{column}_max': max_value,
        f'{column}_max_info': max_info
    })

In [ ]:
min_max_experience = df_top_six.groupby(['Player Position', 'Role']).apply(
    lambda df: pd.concat([
        min_max_with_season_player(df, 'Top Competition Starts Before Season'),
        min_max_with_season_player(df, 'Top Competition Appearances Before Season'),
        min_max_with_season_player(df, 'Total Starts Before Season'),
        min_max_with_season_player(df, 'Total Appearances Before Season')
    ])
).reset_index()

In [ ]:
min_max_experience

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
min_max_experience_file = os.path.join(output_directory, 'min_max_experience.csv')
min_max_experience.to_csv(min_max_experience_file, index=False)
print(f"Min-max experience saved to {min_max_experience_file}")

#### Initial experience

In [ ]:
df_top_six_sorted = df_top_six.sort_values(by=['Player URL', 'Season'])
df_top_six_initial_experience = df_top_six_sorted.drop_duplicates(subset='Player URL', keep='first')

In [ ]:
df_top_six_initial_experience

In [ ]:
df_top_six_initial_experience.columns

In [ ]:
total_apps_initial_experience = df_top_six_initial_experience.groupby(['Player Position', 'Role'])['Total Appearances Before Season'].mean().reset_index()

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

plt.figure(figsize=(16, 10))

barplot = sns.barplot(
    data=total_apps_initial_experience, 
    x='Player Position', 
    y='Total Appearances Before Season', 
    hue='Role',
    order=position_order
)

for p in barplot.patches:
    height = p.get_height()
    if not np.isnan(height):
        barplot.annotate(format(height, '.2f'), 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha='center', va='center', 
                        xytext=(0, 10), 
                        textcoords='offset points')

plt.title('Average Initial Total Appearances Before Season by Role and Position')
plt.xlabel('Player Position')
plt.ylabel('Average Appearances')
plt.legend(title='Role')
plt.xticks(rotation=45)
plt.show()

In [ ]:
total_starts_initial_experience = df_top_six_initial_experience.groupby(['Player Position', 'Role'])['Total Starts Before Season'].mean().reset_index()

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

plt.figure(figsize=(16, 10))

barplot = sns.barplot(
    data=total_starts_initial_experience, 
    x='Player Position', 
    y='Total Starts Before Season', 
    hue='Role',
    order=position_order
)

for p in barplot.patches:
    height = p.get_height()
    if not np.isnan(height):
        barplot.annotate(format(height, '.2f'), 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha='center', va='center', 
                        xytext=(0, 10), 
                        textcoords='offset points')

plt.title('Average Initial Total Appearances Before Season by Role and Position')
plt.xlabel('Player Position')
plt.ylabel('Average Appearances')
plt.legend(title='Role')
plt.xticks(rotation=45)
plt.show()

### Min-max experience (filtered)

In [ ]:
min_max_experience_filtered = df_top_six.groupby(['Player Position', 'Role']).apply(
    lambda df: pd.concat([
        min_max_with_season_player(df, 'Top Competition Starts Before Season'),
        min_max_with_season_player(df, 'Top Competition Appearances Before Season'),
        min_max_with_season_player(df, 'Total Starts Before Season'),
        min_max_with_season_player(df, 'Total Appearances Before Season')
    ])
).reset_index(drop=True)

min_max_experience_filtered.columns = [' '.join(col).strip() for col in min_max_experience_filtered.columns.values]

In [ ]:
min_max_experience_filtered

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
min_max_experience_file = os.path.join(output_directory, 'min_max_experience.csv')
min_max_experience.to_csv(min_max_experience_file, index=False)
print(f"Min-max experience saved to {min_max_experience_file}")

#### Squad consistency

In [ ]:
relevant_roles = df_top_six[df_top_six['Role'].isin(['1st choice', '2nd choice'])]

player_tenure = relevant_roles.groupby(['Team', 'Player Name']).size().reset_index(name='Tenure')

team_consistency = player_tenure.groupby('Team')['Tenure'].mean().reset_index()

team_consistency = team_consistency.sort_values('Tenure', ascending=False).reset_index(drop=True)

print(team_consistency)

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
team_consistency_file = os.path.join(output_directory, 'team_consistency.csv')
team_consistency.to_csv(team_consistency_file, index=False)
print(f"Team consistency saved to {team_consistency_file}")

In [ ]:
player_tenure_sorted = player_tenure.sort_values('Tenure', ascending=False).reset_index(drop=True)

player_tenure_sorted.head(20)

In [ ]:
relevant_roles = df_top_six[df_top_six['Role'].isin(['1st choice', '2nd choice'])]

player_tenure = relevant_roles.groupby(['Player Name', 'Player Position', 'Player URL']).size().reset_index(name='Tenure')

df_top_six = df_top_six.merge(player_tenure[['Player URL', 'Tenure']], on='Player URL', how='left')

df_top_six['Tenure'] = df_top_six['Tenure'].fillna(0)

In [ ]:
position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']

average_tenure_by_position = df_top_six.groupby('Player Position')['Tenure'].mean().reset_index()

In [ ]:
average_tenure_by_position

In [ ]:
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Tenure', y='Player Position', data=average_tenure_by_position, order=position_order, palette="Blues_d")

plt.xlabel('Average Tenure')
plt.ylabel('Player Position')
plt.title('Average Tenure by Position in Top Six Teams')

for p in barplot.patches:
    width = p.get_width()
    plt.text(p.get_x() + width + 0.11,
             p.get_y() + p.get_height()/2,
             '{:1.2f}'.format(width),
             ha = 'center',
             va = 'center')

#### Chelsea!

In [ ]:
csv_file_path = 'season_summaries\\2023-2024_top_players.csv'

df_chelsea = pd.read_csv(csv_file_path)

chelsea_df = df_chelsea[df_chelsea['Team'] == 'Chelsea FC']

In [ ]:
chelsea_df.columns

In [ ]:
chelsea_df = chelsea_df.drop(['League Position',
                              'Player URL',
                              'Team',
                              'Season',
                              'Role'], axis=1)

In [ ]:
chelsea_df

In [ ]:
output_directory = 'analysis_output/csv'
os.makedirs(output_directory, exist_ok=True)

In [ ]:
chelsea_df_file = os.path.join(output_directory, 'chelsea_current_state.csv')
chelsea_df.to_csv(chelsea_df_file, index=False)
print(f"Chelsea current squad state saved to {chelsea_df_file}")

In [ ]:
average_values = chelsea_df[[
    'Age at End of Season',
    'PL Starts This Season', 
    'PL Appearances This Season',
    'Top Competition Starts Before Season',
    'Top Competition Appearances Before Season',
    'Total Starts Before Season', 
    'Total Appearances Before Season'
]].mean()

average_values = average_values.round(2)

print(average_values)

In [ ]:
chelsea_df_filtered = chelsea_df[
    (chelsea_df['Player Name'] != 'Raheem Sterling') &
    (chelsea_df['Player Name'] != 'Thiago Silva')
]

average_values = chelsea_df_filtered[[
    'Age at End of Season',
    'PL Starts This Season', 
    'PL Appearances This Season',
    'Top Competition Starts Before Season',
    'Top Competition Appearances Before Season',
    'Total Starts Before Season', 
    'Total Appearances Before Season'
]].mean()

average_values = average_values.round(2)

print(average_values)

In [ ]:
csv_file_path = 'season_summaries\\2023-2024_top_players.csv'

df_rest_of_the_league = pd.read_csv(csv_file_path)

rest_of_the_league_df = df_rest_of_the_league[df_rest_of_the_league['Team'] != 'Chelsea FC']

In [ ]:
rest_of_the_league_df

In [ ]:
average_values = rest_of_the_league_df[[
    'Age at End of Season',
    'PL Starts This Season', 
    'PL Appearances This Season',
    'Top Competition Starts Before Season',
    'Top Competition Appearances Before Season',
    'Total Starts Before Season', 
    'Total Appearances Before Season'
]].mean()

average_values = average_values.round(2)

print(average_values)